In [1]:
from transformers import pipeline
#https://huggingface.co/diwank/silicone-deberta-pair
model = "diwank/silicone-deberta-pair"
tokenizer = "diwank/silicone-deberta-pair"
pip = pipeline('text-classification', model=model)

In [2]:
# (0, 'acknowledge')
# (1, 'answer')
# (2, 'backchannel')
# (3, 'reply_yes')
# (4, 'exclaim')
# (5, 'say')
# (6, 'reply_no')
# (7, 'hold')
# (8, 'ask')
# (9, 'intent')
# (10, 'ask_yes_no')

In [4]:
sentences = ["I love cats", "Do you love cats?", "Yes, I do", "No, dogs"]

In [6]:
response = pip(sentences)
print(response)

[{'label': 'LABEL_5', 'score': 0.7644516229629517}, {'label': 'LABEL_8', 'score': 0.9776815176010132}, {'label': 'LABEL_3', 'score': 0.5020651817321777}, {'label': 'LABEL_5', 'score': 0.5964227318763733}]


In [ ]:
#https://github.com/DianDYu/MIDAS_dialog_act

In [ ]:
python run_classifier.py --data_dir da_data/ --bert_model output --task_name da --output_dir output --do_eval --binary_pred